# PrimoGPT NLP Features Evaluation

This notebook evaluates the predictive accuracy of NLP features generated by PrimoGPT model. The evaluation compares generated features against actual market movements in the following period.

## Evaluation Period
- Start Date: January 1, 2024
- End Date: July 31, 2024
- Purpose: Aligns with PrimoRL trading simulation phase

## Feature Evaluation
The notebook analyzes the predictive power of:
1. Combined evaluation algorithm (sentiment + trend)
2. Individual features:
   - News Relevance (0-2)
   - Sentiment (-1 to 1)
   - Price Impact Potential (-3 to 3)
   - Trend Direction (-1 to 1)
   - Earnings Impact (-2 to 2)
   - Investor Confidence (-3 to 3)
   - Risk Profile Change (-2 to 2)

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

In [2]:
csv_file_name = "data/NFLX_data.csv"

df = pd.read_csv(csv_file_name)

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filter data for the specified date range
start_date = '2024-01-01'
end_date = '2024-07-31'
df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

df.head()

,Date,Adj Close Price,Returns,Bin Label,News Relevance,Sentiment,Price Impact Potential,Trend Direction,Earnings Impact,Investor Confidence,Risk Profile Change,Prompt
438,2024-01-02,468.500000,-0.037751,D4,1,0,-1,-1,0,-1,0,\n [COMPANY BASICS]\n ...
439,2024-01-03,470.260010,0.003757,U1,1,0,0,0,0,0,0,\n [COMPANY BASICS]\n ...
440,2024-01-04,474.670013,0.009378,U1,2,1,1,1,1,2,0,\n [COMPANY BASICS]\n ...
441,2024-01-05,474.059998,-0.001285,D1,2,0,0,0,1,1,0,\n [COMPANY BASICS]\n ...
442,2024-01-08,485.029999,0.023141,U3,2,0,1,1,0,1,0,\n [COMPANY BASICS]\n ...


In [3]:
def predict_movement(row, previous_direction):
    if row['News Relevance'] == 0 or (row['Sentiment'] + row['Trend Direction'] == 0):
        return previous_direction
    
    score = row['Sentiment'] + row['Trend Direction']
    
    if score > 0:
        return 'U'
    else:
        return 'D'

def classify_actual_movement(bin_label):
    if pd.isna(bin_label) or bin_label is None:
        return 'U'  # Assume 'Up' for missing values (or you can choose 'D')
    bin_label = str(bin_label)  # Convert to string just in case
    if bin_label.startswith('U'):
        return 'U'
    else:
        return 'D'

# Add actual values for the current day
df['Actual_Current_Day'] = df['Bin Label'].apply(classify_actual_movement)

# Initialize a list for predictions
predictions = []

# Iterate through the DataFrame and predict for each row
previous_direction = 'U'  # Initial assumption for the first row
for _, row in df.iterrows():
    prediction = predict_movement(row, previous_direction)
    predictions.append(prediction)
    previous_direction = row['Actual_Current_Day']  # Update previous_direction for the next iteration

# Add predictions to the DataFrame
df['Predicted_Next_Day'] = predictions

# Shift actual values one day forward for comparison with predictions
df['Actual_Next_Day'] = df['Actual_Current_Day'].shift(-1)

# Remove the last row since we don't have an actual value for the next day after the last day
df = df.dropna(subset=['Actual_Next_Day'])

# Calculate accuracy
accuracy = accuracy_score(df['Actual_Next_Day'], df['Predicted_Next_Day'])
conf_matrix = confusion_matrix(df['Actual_Next_Day'], df['Predicted_Next_Day'], labels=['U', 'D'])

print(f"Točnost predviđanja: {accuracy:.2f}")
print("\nMatrica zabune:")
print(pd.DataFrame(conf_matrix, index=['Actual U', 'Actual D'], columns=['Pred U', 'Pred D']))

print("\nIzvještaj o klasifikaciji:")
print(classification_report(df['Actual_Next_Day'], df['Predicted_Next_Day']))

# Analyze the accuracy of the combination of sentiment and trend
correct_predictions = ((df['Sentiment'] + df['Trend Direction'] > 0) & (df['Actual_Next_Day'] == 'U')) | \
                      ((df['Sentiment'] + df['Trend Direction'] < 0) & (df['Actual_Next_Day'] == 'D')) | \
                      ((df['Sentiment'] + df['Trend Direction'] == 0) & (df['Predicted_Next_Day'] == df['Actual_Next_Day']))
combined_accuracy = correct_predictions.mean()
print(f"\nTočnost kombinacije Sentiment + Trend Direction: {combined_accuracy:.2f}")

Točnost predviđanja: 0.46

Matrica zabune:
          Pred U  Pred D
Actual U      44      27
Actual D      51      23

Izvještaj o klasifikaciji:
              precision    recall  f1-score   support

           D       0.46      0.31      0.37        74
           U       0.46      0.62      0.53        71

    accuracy                           0.46       145
   macro avg       0.46      0.47      0.45       145
weighted avg       0.46      0.46      0.45       145


Točnost kombinacije Sentiment + Trend Direction: 0.46
